# 이론
#### 경사하강법
**경사하강법**이란 미분 가능한 함수의 최솟값을 근사하기 위한 알고리즘의 일종.   
최적의 기울기를 찾는 것이 목표이다. 실제로는 많은 수의 가중치 및 편향에 대해 최적화를 해야하며 손실 함수도 복잡하다.   
극값에 가까워질수록 점점 조금씩 움직여야 제대로 도달한다.   
학습률이 너무 높으면 수렴하지 못하고, 너무 낮으면 국소 최솟값(local minimal)에 빠지게 된다.   
#### 다중 선형회귀   
앞선 선형회귀에서는 독립변수가 하나였지만, 다양한 독립변수들에 의해 y가 결정될 수 있다.   
다만 여전히 독립변수들과 y의 관계를 선형으로 가정하는 것은 똑같다. 이러한 여러 독립변수들에 대한 선형회귀를 **다중 선형회귀**라고 한다.   
다중 선형회귀에서는 벡터 형태의 W 및 스칼라 형태의 b로 변수들을 생성해 행렬 형태의 학습 데이터에 대해 **행렬 연산으로 간편히 예측값을 계산**할 수 있다. 이렇게 구현치 않으면 각 W에 대해 변수를 생성하는 등 번거롭다.

In [1]:
import torch
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 80],
                             [96, 98, 100]])
y_train = torch.FloatTensor([[152], [185], [180], [196]])

#### torch.nn.Module   
토치에서 모든 신경망 모델의 base class이다. 그렇기에 토치에서 모든 신경망 클래스를 구현할 때는 이를 상속받아야 한다.
##### nn.Linear
nn.Linear는 fully connected neural network를 구성할 때 사용하는 클래스로, **nn.Linear(입력 크기, 출력 크기)** 형태로 호출한다.
#### forward
해당 모델에 **입력이 들어왔을 때 어떻게 계산하여 결과값을 만들고 반환할지를 명시**하는 메서드이다.   오버라이딩이기 때문에 원하는대로 설정이 가능하다.   
변수명(입력값)형태로 호출하면 forward 메서드가 호출된다.
- ex. self.linear(x)는 self.linear객체의 forward 메서드를 호출하는 셈.

In [2]:
import torch.nn as nn

class MLRM(nn.Module): #MultipleLinearRegressionModel
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)
  def forward(self, x):
    return self.linear(x)

#### nn.functional
대표적인 손실 함수를 비롯한 다양한 함수들을 포함한다.   
예를 들어 MSE같은 경우 mse_loss라는 이름의 함수로 존재하며 사용시 mse_loss(예측값, 레이블)형태로 호출한다.   
nn.Module에서 parameters() 메서드를 호출하면 내부의 모든 가중치 및 편향을 모아 반환한다.   
이전에서 W, b를 직접 선언하는 것과 달리 nn.Module을 이용한 클래스 형태로 모델을 구현하면 optimizer 선언 시 **parameters() 메서드를 활용해 최적화를 진행할 변수들을 입력**한다.   
토치에서는 가중치 및 편향을 초기화하는 디폴트 방식이 존재해 초기화된 상태는 zero tensor가 아니다.

In [5]:
import torch.nn.functional as F

model = MLRM()
print(list(model.parameters()))

optimizer = torch.optim.SGD(model.parameters(), lr=1e-5)
nb_epochs = 10000
for epoch in range(nb_epochs+1):
  prediction = model(x_train)
  cost = F.mse_loss(prediction, y_train)

  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  if epoch % 1000 == 0:
    print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()
    ))

test_data = torch.FloatTensor([[73, 66, 70]])
prediction = model(test_data)
print(prediction)

[Parameter containing:
tensor([[-0.4129, -0.1977, -0.3570]], requires_grad=True), Parameter containing:
tensor([-0.4013], requires_grad=True)]
Epoch    0/10000 Cost: 70061.468750
Epoch 1000/10000 Cost: 4.763116
Epoch 2000/10000 Cost: 3.236203
Epoch 3000/10000 Cost: 2.262374
Epoch 4000/10000 Cost: 1.612404
Epoch 5000/10000 Cost: 1.163724
Epoch 6000/10000 Cost: 0.846741
Epoch 7000/10000 Cost: 0.619365
Epoch 8000/10000 Cost: 0.454637
Epoch 9000/10000 Cost: 0.334583
Epoch 10000/10000 Cost: 0.246747
tensor([[142.0727]], grad_fn=<AddmmBackward0>)
